In [16]:
'''ANL moodel'''

import torch
import torch.nn as nn


class AdvNoise(nn.Module):
    def __init__(self, eps):
        super(AdvNoise, self).__init__()

        self.adv = None
        self.eps = eps
        self.input_std = 0.0
        self.dim = None

        self.status = 'clean'

    def extra_repr(self):
        return 'eps=%f' % (self.eps)

    def set_clean(self):
        self.status = 'clean'

    def set_stay(self):
        self.status = 'stay'

    def zero_adv(self):
        if self.adv is not None:
            self.adv.data.zero_()

    def cal_r(self, r):
        r.normal_()
        r = torch.clamp(r * (abs(self.eps) / 4) + self.eps / 2, min(0, self.eps), max(0, self.eps)) * self.input_std
        return r

    def update_adv(self, grad):
        if self.training and self.status == 'clean':
            n = grad.data.view(self.adv.shape[0], -1).max(dim=1)[0].view(*self.dim)
            r = self.cal_r(torch.zeros_like(n))
            # self.adv.data.set_(r * grad.data / (1e-6 + n))
            with torch.no_grad():
              self.adv.set_(r * grad / (1e-6 + n))

    # pylint: disable=W0221,E1101
    def forward(self, x):
        if self.eps == 0.0:
            return x

        if self.adv is None:
            self.adv = torch.zeros_like(x, requires_grad=True)
            self.adv.register_hook(self.update_adv)
 
            self.dim = [1 if i != 0 else -1 for i in range(0, len(self.adv.shape))]

        if not self.training or x.shape != self.adv.shape:
            return x

        if self.status == 'clean':
            self.zero_adv()

        if self.status == 'stay':
            self.input_std = 0.9 * self.input_std + 0.1 * x.std().data.item()

        return x + self.adv


In [5]:
from google.colab import files
uploaded = files.upload()

Saving vgg.json to vgg.json


In [6]:
 '''config'''
import argparse
import json
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

def make_as_dotdict(obj):
    if type(obj) is dict:
        obj = dotdict(obj)
        for key in obj:
            if type(obj[key]) is dict:
                obj[key] = make_as_dotdict(obj[key])
    return obj

def parse():
    print('Parsing config file...')
    parser = argparse.ArgumentParser(description="config")
    parser.add_argument(
        "--config",
        type=str,
        default="vgg.json",
        help="Configuration file to use"
    )
    cli_args = parser.parse_args()

    with open(cli_args.config) as fp:
        config = make_as_dotdict(json.loads(fp.read()))
    print(json.dumps(config, indent=4, sort_keys=True))
    return config

with open("vgg.json") as fp:
        config = make_as_dotdict(json.loads(fp.read()))
cfg = config

# if cfg is None:
#     try:
#         cfg = parse()
#     except:
#         print('** Assert in demo mode. **')

def parse_from_dict(d):
    global cfg
    cfg = make_as_dotdict(d)


In [7]:
'''loader'''
import torch
import torchvision
from torchvision import transforms

def get_cifar10():
    '''
        get dataloader
    '''
    print('==> Preparing Cifar10 data..')
    transform_train = transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
    ])

    transform_test = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.4914, 0.4822, 0.4465), (0.247, 0.243, 0.261)),
    ])

    trainset = torchvision.datasets.CIFAR10(
        root='./data', train=True, download=True, transform=transform_train)
    train_loader = torch.utils.data.DataLoader(
        trainset, batch_size=cfg.data.batch_size, shuffle=cfg.data.shuffle, num_workers=cfg.data.num_workers)

    testset = torchvision.datasets.CIFAR10(
        root='./data', train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(
        testset, batch_size=cfg.data.test_batch_size, shuffle=False, num_workers=cfg.data.num_workers)

    return train_loader, test_loader

In [8]:
'''trainer'''

def accuracy(output, target, topk=(1,)):
    """Computes the accuracy over the k top predictions for the specified values of k"""
    with torch.no_grad():
        maxk = max(topk)
        batch_size = target.size(0)

        _, pred = output.topk(maxk, 1, True, True)
        pred = pred.t()
        correct = pred.eq(target.view(1, -1).expand_as(pred))

        res = []
        for k in topk:
            correct_k = correct[:k].view(-1).float().sum(0, keepdim=True)
            res.append(correct_k.mul_(100.0 / batch_size))
        return res


class Trainer():
    def __init__(self):
        self.use_cuda = cfg.base.cuda

    def test(self, pack, topk=(1,)):
        pack.net.eval()
        loss_acc = 0.0
        hub = [[] for i in range(len(topk))]

        for data, target in pack.test_loader:
            if self.use_cuda:
                data, target = data.cuda(), target.cuda()

            with torch.no_grad():
                output = pack.net(data)
                loss_acc += pack.criterion(output, target).data.item()
                acc = accuracy(output, target, topk)
                for acc_idx, score in enumerate(acc):
                    hub[acc_idx].append(score[0].item())

        loss_acc /= len(pack.test_loader)
        info = {
            'test_loss': loss_acc
        }

        for acc_idx, k in enumerate(topk):
            info['acc@%d' % k] = np.mean(hub[acc_idx])

        return info

    def train(self, pack):
        pack.net.train()
        loss_acc = 0.0
        begin = time()

        pack.optimizer.zero_grad()
        with tqdm(total=len(pack.train_loader), ncols=100) as pbar:
            for _, (data, label) in enumerate(pack.train_loader):
                if self.use_cuda:
                    data, label = data.cuda(), label.cuda()
                data = Variable(data, requires_grad=False)
                label = Variable(label)

                pack.net.apply(lambda m: type(m) == AdvNoise and m.set_clean())
                pack.optimizer.zero_grad()
                est_out = pack.net(data)
                loss = pack.criterion(est_out, label)
                loss.backward()

                pack.net.apply(lambda m: type(m) == AdvNoise and m.set_stay())
                pack.optimizer.zero_grad()
                out = pack.net(data)
                loss = pack.criterion(out, label)

                loss.backward()
                pack.optimizer.step()

                loss_acc += loss.item()
                pbar.update(1)

        info = {
            'train_loss': loss_acc / len(pack.train_loader),
            'epoch_time': time() - begin
        }
        return info


In [9]:
_cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}


class VGG(nn.Module):
    def __init__(self, vgg_name, anl_creator, num_class):
        super(VGG, self).__init__()

        self.anl_creator = anl_creator

        self.features = self._make_layers(_cfg[vgg_name])
        self.classifier = nn.Linear(512, num_class)

    # pylint: disable=W0221,E1101
    def forward(self, x):
        out = self.features(x)
        out = out.view(out.size(0), -1)
        out = self.classifier(out)
        return out

    def _make_layers(self, cfg):
        layers = []
        in_channels = 3
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                layers += [nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                           nn.BatchNorm2d(x),
                           self.anl_creator(),
                           nn.ReLU(inplace=True)]
                in_channels = x
        layers += [nn.AvgPool2d(kernel_size=1, stride=1)]
        return nn.Sequential(*layers)

In [10]:
import torch

def get_anl_creator(eps):
    def anl_creator():
        return AdvNoise(eps=eps)
    return anl_creator

def get_vgg():
    return VGG('VGG16', get_anl_creator(cfg.anl.eps), 10)

def get_mobilenetv2():
    from models.mobilenetv2 import MobileNetV2
    return MobileNetV2(get_anl_creator(cfg.anl.eps), 10)

def get_wide_resnet():
    from models.wide_resnet import WideResNet
    return WideResNet(get_anl_creator(cfg.anl.eps), 28, 10, 10, 0.0)

def get_model():
    pair = {
        'vgg16': get_vgg,
        'mobileNetV2': get_mobilenetv2,
        'wide_resnet': get_wide_resnet
    }

    model = pair[cfg.model.name]()

    if cfg.base.cuda:
        model = model.cuda()

    if cfg.base.multi_gpus:
        model = torch.nn.DataParallel(model)

    return model


In [ ]:
'''main'''
from time import time
from tqdm import tqdm
from torch.autograd import Variable

import torch
import torch.nn as nn
import torch.optim as optim

import numpy as np
import random
import math



def _step_lr(epoch):
    v = 0.0
    for max_e, lr_v in cfg.train.steplr:
        v = lr_v
        if epoch <= max_e:
            break
    return v


def get_lr_func():
    return _step_lr


def adjust_learning_rate(epoch, pack):
    if pack.optimizer is None:
        if cfg.train.optim == 'sgd' or cfg.train.optim is None:
            pack.optimizer = optim.SGD(
                pack.net.parameters(),
                lr=1,
                momentum=cfg.train.momentum,
                weight_decay=cfg.train.weight_decay,
                nesterov=cfg.train.nesterov
            )
        else:
            print('WRONG OPTIM SETTING!')
            assert False
        pack.lr_scheduler = optim.lr_scheduler.LambdaLR(
            pack.optimizer, get_lr_func())
        if cfg.base.fp16 and cfg.base.cuda:
            from apex.fp16_utils import FP16_Optimizer
            pack.optimizer = FP16_Optimizer(
                pack.optimizer, dynamic_loss_scale=True)

    pack.lr_scheduler.step(epoch)
    return pack.lr_scheduler.get_lr()


def recover_pack():
    train_loader, test_loader = get_cifar10()

    pack = dotdict({
        'net': get_model(),
        'train_loader': train_loader,
        'test_loader': test_loader,
        'trainer': Trainer(),
        'criterion': nn.CrossEntropyLoss(),
        'optimizer': None,
        'lr_scheduler': None
    })

    adjust_learning_rate(cfg.base.epoch, pack)
    return pack


def set_seeds():
    torch.manual_seed(cfg.base.seed)
    if cfg.base.cuda:
        torch.cuda.manual_seed_all(cfg.base.seed)
        torch.backends.cudnn.deterministic = True
    np.random.seed(cfg.base.seed)
    random.seed(cfg.base.seed)


def main():
    set_seeds()
    pack = recover_pack()

    for epoch in range(cfg.base.epoch + 1, cfg.train.max_epoch + 1):
        lr = adjust_learning_rate(epoch, pack)
        info = pack.trainer.train(pack)
        info.update(pack.trainer.test(pack))
        info.update({'LR': lr})
        print(epoch, info)


main()


==> Preparing Cifar10 data..
Files already downloaded and verified
Files already downloaded and verified


/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)
/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:143: UserWarning: The epoch parameter in `scheduler.step()` was not necessary and is being deprecated where possible. Please use `scheduler.step()` to step the scheduler. During the deprecation, if epoch is different from None, the closed form is used instead of the new chainable form, where available. Please open an issue if you are unable to replicate your use case: https://git

1 {'train_loss': 2.5188497862852444, 'epoch_time': 67.0630612373352, 'test_loss': 2.2682167053222657, 'acc@1': 12.5390625, 'LR': [0.1]}


  0%|                                                                       | 0/391 [00:00<?, ?it/s]

2 {'train_loss': 2.2796479649555956, 'epoch_time': 66.52297592163086, 'test_loss': 2.240351903438568, 'acc@1': 14.356617641448974, 'LR': [0.1]}


...

  0%|                                                                       | 0/391 [00:00<?, ?it/s]

85 {'train_loss': 0.27250242450505574, 'epoch_time': 66.91165590286255, 'test_loss': 0.19819305837154388, 'acc@1': 93.2858455657959, 'LR': [0.01]}


  0%|                                                                       | 0/391 [00:00<?, ?it/s]

86 {'train_loss': 0.2571043655504961, 'epoch_time': 66.91115260124207, 'test_loss': 0.1967231310904026, 'acc@1': 93.38235282897949, 'LR': [0.01]}


 28%|█████████████████▏                                           | 110/391 [00:19<00:47,  5.87it/s]